In [ ]:
import plotly.express as px
import pandas as pd
from lib.utils import load_delta_dir_by_path,run_old_state_machine_on_thresholded_predictions,run_new_state_machine_on_thresholded_predictions,forward_casey,forward_casey_corrected
from lib.models import MLP
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df,X = load_delta_dir_by_path(f'/home/andrew/com.example.delta/files/2022-12-07_16_18_18/raw/2022-12-07_16_18_18.0.csv')
df = df[:20000]
X = X[:19901]

In [ ]:
# let's visualize the data
fig = px.line(df,x='index',y=['acc_x','acc_y','acc_z'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [ ]:
# let's visualize the raw network output
fig = px.line(df,x='index',y=['acc_x','acc_y','acc_z','rawlabel_10'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [ ]:
# how does this translate into puff detection, and therefore session detection?
# first we visualize thresholded rawlabels
fig = px.line(df,x='index',y=['acc_x','acc_y','acc_z','rawlabel_10','label_10'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [ ]:
# now we can get rid of rawlabels, understanding that the algorithm starts there. let's get rid of y and z also for now.
fig = px.line(df,x='index',y=['acc_x','label_10'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [ ]:
# how do thresholded outputs translate into decisions about puffs?
fig = px.line(df,x='index',y=['acc_x','label_10','state_10'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [ ]:
# recreate old pipeline
rawlabel_old = forward_casey(X)
df['rawlabel_old'] = rawlabel_old
df['rawlabel_old_10'] = df['rawlabel_old']*10
# do thresholding
label_old = [1 if y > .85 else 0 for y in rawlabel_old]
df['label_old'] = label_old
df['label_old_10'] = df['label_old']*10
# run old state machine
states_old,puff_locations_old = run_old_state_machine_on_thresholded_predictions(label_old)
df['state_old'] = states_old
# let's plot puff locations
fig = px.line(df,y=['acc_x','label_old_10','state_old'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations_old:
    fig.add_vline(x=puff_loc)
fig.show(renderer='browser')

In [ ]:
# recreate new pipeline
rawlabel = forward_casey_corrected(X)
# verify
# rawlabel == df['rawlabel']
# np.allclose(rawlabel[:19004],df['rawlabel'][:19004])
# np.allclose(rawlabel,df['rawlabel'])
# plt.plot(rawlabel[-100:])
# plt.plot(df['rawlabel'].to_numpy()[-100:])
# thresholding
label = [1 if y > .85 else 0 for y in rawlabel]
(label == df['label']).all() # works because integer-valued
# state machine
states,puff_locations = run_new_state_machine_on_thresholded_predictions(label)
(states == df['state']).all() # works because integer-valued
# compare old and new
fig = px.line(df,y=['acc_x','label_10','state','label_old_10','state_old'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc,line_color="green")
for puff_loc in puff_locations_old:
    fig.add_vline(x=puff_loc, line_color="red",line_dash="dash",line_width=2)
fig.show(renderer='browser')

In [ ]:
# session detection
puffs = np.zeros(len(df))
puffs[np.array(puff_locations)] = 1
puffs_in_session = []
for i,puff in enumerate(puffs):
    if i < 9600:
        puffs_in_session.append(puffs[0:i+1].sum())
    else:
        puffs_in_session.append(puffs[i-9599:i+1].sum())
df['puff_count'] = puffs_in_session

In [ ]:
session_idx = []
for i,(puff,puff_count) in enumerate(zip(puffs,puffs_in_session)):
    if(puff and puff_count > 2):
        session_idx.append(i)

In [ ]:
# session detection
puffs = np.zeros(len(df))
puffs[np.array(puff_locations_old)] = 1
puffs_in_session = []
for i,puff in enumerate(puffs):
    if i < 9600:
        puffs_in_session.append(puffs[0:i+1].sum())
    else:
        puffs_in_session.append(puffs[i-9599:i+1].sum())
df['puff_count_old'] = puffs_in_session

In [123]:
# session detection
puff_signal = np.zeros(len(df))
puff_signal[np.array(puff_locations_old)] = 1
puffs_in_session = []
session_count = 0
sessions = []
puffs = []
puff_count = 0
session = False
in_session = []
puff_counts = []
for i,_ in enumerate(puff_signal):
    puffs = [puff - 1 for puff in puffs]
    if (session):
        if(puffs[0] == 0):
            # meaning the earliest puff in the session has now expired
            session = False
        else:
            puffs_in_session.append(len(puffs))
            puff_counts.append(puff_count)
            sessions.append(session_count)
            in_session.append(1)
            continue
    puffs = [puff for puff in puffs if puff > 0]
    if(puff_signal[i]):
        puff_count += 1
        puffs.append(9600)
    if(puff_count == 3):
        puff_count = 0
        session_count += 1
        session = True
        
    # if i < 9600:
    #     puffs_in_session.append(puffs[0:i+1].sum())
    # else:
    #     puffs_in_session.append(puffs[i-9599:i+1].sum())
    puffs_in_session.append(len(puffs))
    in_session.append(0)
    puff_counts.append(puff_count)
    sessions.append(session_count)
df['puff_count_old'] = puffs_in_session
df['puff_counts'] = puff_counts
df['sessions'] = sessions
df['in_session'] = in_session
fig = px.line(df,y=['acc_x','puff_count_old','puff_counts','sessions','in_session'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations_old:
    fig.add_vline(x=puff_loc, line_color="red",line_width=1)
fig.show(renderer='browser')

In [ ]:
puffs

In [ ]:
session_idx_old = []
for i,(puff,puff_count) in enumerate(zip(puffs,puffs_in_session)):
    if(puff and puff_count > 2):
        session_idx_old.append(i)

In [ ]:
fig = px.line(df[:50000],y=['acc_x','puff_count','puff_count_old'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')


In [ ]:

for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc,line_dash="dash", line_color="green",line_width=.3)
for puff_loc in puff_locations_old:
    fig.add_vline(x=puff_loc, line_color="red",line_width=.3)
for session_loc in session_idx:
    fig.add_vline(x=session_loc,line_dash="dash", line_color="green",line_width=2)
for session_loc in session_idx_old:
    fig.add_vline(x=session_loc, line_color="red",line_width=2)
fig.show(renderer='browser')